# Day 8: I Heard You Like Registers

You receive a signal directly from the CPU. Because of your recent assistance with jump instructions, it would like you to compute the result of a series of unusual register instructions.

Each instruction consists of several parts: the register to modify, whether to increase or decrease that register's value, the amount by which to increase or decrease it, and a condition. If the condition fails, skip the instruction without modifying the register. The registers all start at 0.

In [1]:
from collections import defaultdict
import operator


def parse(s):
    insts = []
    for line in s.strip().splitlines():
        cmd, cond = [w.split() for w in line.strip().split(' if ')]
        cmd = {'reg': cmd[0], 'op': cmd[1], 'val': int(cmd[2])}
        cond = {'reg': cond[0], 'cmp': cond[1], 'val': int(cond[2])}
        inst = {'cmd': cmd, 'cond': cond}
        insts.append(inst)
    return insts


OP = {
    'inc': lambda x, y: x + y,
    'dec': lambda x, y: x - y,
}

CMP = {
    '==': operator.eq,
    '!=': operator.ne,
    '>': operator.gt,
    '<': operator.lt,
    '>=': operator.ge,
    '<=': operator.le
}


def run(insts):
    regs = defaultdict(lambda: 0)
    yield regs
    
    for inst in insts:
        # skip if condition is not met
        cond = inst['cond']
        if not CMP[cond['cmp']](regs[cond['reg']], cond['val']):
            continue

        # execute operation
        cmd = inst['cmd']
        regs[cmd['reg']] = OP[cmd['op']](regs[cmd['reg']], cmd['val'])
        yield regs

def highest_value(insts):
    for regs in run(insts):
        pass
    return max(regs.values())

The instructions look like this:

In [2]:
example = parse("""
b inc 5 if a > 1
a inc 1 if b < 5
c dec -10 if a >= 1
c inc -20 if c == 10
""")

[{'cmd': {'op': 'inc', 'reg': 'b', 'val': 5},
  'cond': {'cmp': '>', 'reg': 'a', 'val': 1}},
 {'cmd': {'op': 'inc', 'reg': 'a', 'val': 1},
  'cond': {'cmp': '<', 'reg': 'b', 'val': 5}},
 {'cmd': {'op': 'dec', 'reg': 'c', 'val': -10},
  'cond': {'cmp': '>=', 'reg': 'a', 'val': 1}},
 {'cmd': {'op': 'inc', 'reg': 'c', 'val': -20},
  'cond': {'cmp': '==', 'reg': 'c', 'val': 10}}]

These instructions would be processed as follows:

- Because `a` starts at 0, it is not greater than 1, and so `b` is not modified.
- `a` is increased by 1 (to 1) because `b` is less than 5 (it is 0).
- `c` is decreased by -10 (to 10) because `a` is now greater than or equal to 1 (it is 1).
- `c` is increased by -20 (to -10) because `c` is equal to 10.

After this process, the largest value in any register is 1.

In [3]:
assert highest_value(example) == 1

You might also encounter `<=` (less than or equal to) or `!=` (not equal to). However, the CPU doesn't have the bandwidth to tell you what all the registers are named, and leaves that to you to determine.

What is the largest value in any register after completing the instructions in your puzzle input?

In [4]:
puzzle = parse(open('08.input').read())
highest_value(puzzle)

5946

## Part Two

To be safe, the CPU also needs to know the highest value held in any register during this process so that it can decide how much memory to allocate to these operations.

In [5]:
def highest_value(insts):
    return max(max(regs.values()) if regs else 0 for regs in run(insts))

For example, in the above instructions, the highest value ever held was 10 (in register `c` after the third instruction was evaluated).

In [6]:
assert highest_value(example) == 10

In [7]:
highest_value(puzzle)

6026